# **Federated learning server without local data sharing**

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import json

import sys
sys.path.append("../")
import neural_network
import utils

import infer_tools
import participant

import concurrent.futures
#from multiprocessing.pool import ThreadPool

from tqdm.notebook import trange

from datetime import datetime

#tf.config.set_visible_devices([], 'GPU')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

## **Preparations**

In [2]:
with open("../../02_data/filtered_vehicles.json", "r") as f:
    filtered_vehicles = json.load(f)
with open("../../dockeroutput/day_0/vehicle_maps.json", "r") as f:
    vehicle_maps = json.load(f)
    veh_to_idx_map = vehicle_maps["vehicle_to_idx_map"]
    idx_to_veh_map = vehicle_maps["idx_to_vehicle_map"]

In [3]:
filtered_vehicles = filtered_vehicles["vehicles"]

In [4]:
settings = {
    "min": 18220,
    "max": 46800,
    "mean": 0.59,
    "std": 0.35302
}

PARKING_ID_LIST = [i for i in range(1059, 1186+1)]

In [5]:
test_t = np.arange(14460, 50400+1, 60)
test_t = utils.normalize(test_t, settings["min"], settings["max"])

In [6]:
oracle = pd.read_csv("../../02_data/oracle.csv")

In [7]:
plist=[i for i in range(1059,1186+1)]
plist.remove(1148)
settings["parkings"] = plist
parking_testset = None
for parking in plist:
    id_encoding = utils.one_hot_encoder([parking]*len(test_t), PARKING_ID_LIST)
    test_d = np.hstack([id_encoding, test_t.reshape(len(test_t), 1)])
    if parking_testset is None:
        parking_testset = test_d
    else:
        parking_testset = np.vstack([parking_testset, test_d])

In [8]:
with open("../../02_data/known_parkings.json", "r") as f:
    known_parkings_by_vehicles = json.load(f)
with open("../../02_data/known_moving_times.json", "r") as f:
    known_moving_times = json.load(f)

## **Learning**

In [9]:
NUM_VEHICLES = 50

In [10]:
participants = [participant.FLParticipant() for i in range(NUM_VEHICLES)]

2023-02-03 16:47:01.361895: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-03 16:47:01.407020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-03 16:47:01.407204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-03 16:47:01.408266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
def train_participant(parameters):
    def _train(participant, weights, veh_num, day):
        return participant.train(weights, veh_num, day, epochs=50)

    def _eval(participant, veh_id, baseline_model, oracle, parking_testset, test_t):
        true_parkings = known_parkings_by_vehicles[veh_id]
        true_time = known_moving_times[veh_id]
        return infer_tools.evaluate_performance(participant.nn.model, oracle, baseline_model, parking_testset,
                                               test_t, settings, true_parkings, true_time)
    
    partip = participants[parameters[0]]
    veh_num = veh_to_idx_map[parameters[2]]
    baseline = parameters[4]
    #print("train")
    weights, samp_num = _train(partip, parameters[1], veh_num, parameters[3])
    #print("eval")
    results = _eval(partip, parameters[2], baseline, parameters[5], parameters[6], parameters[7])
    
    return weights, samp_num, results[0], results[1]

In [12]:
def fed_avg(parameters, weights):
        new_params = []
        for layer_i in range(len(parameters[0])):
            
            new_params.append(
                np.average([parameters[i][layer_i] for i in range(len(parameters))],
                           weights=weights,
                           axis=0
                )
            )
        return new_params

In [13]:
fed_model = neural_network.NeuralNetwork()

fed_weights = fed_model.model.get_weights()

#pool = concurrent.futures.ThreadPoolExecutor(max_workers=16)
#pool = ThreadPool(1)

position_accuracies = []
time_accuracies = []


for day in trange(55):
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("DAY%d\tcurrent Time ="%day, current_time)
    selected_participants = np.random.choice(np.arange(0, len(filtered_vehicles)), NUM_VEHICLES, replace=False)
    selected_vehicles = [filtered_vehicles[i] for i in selected_participants]
    
    params = []
    for i in range(len(selected_vehicles)):
        params.append([i, fed_weights, selected_vehicles[i], day, fed_model.model,
                       oracle, parking_testset, test_t])
    
    weights = []
    num_samps = []
    p_acc, t_acc = [], []
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as pool:
        results = pool.map(train_participant, params)   
        for r in results:
            weights.append(r[0])
            num_samps.append(r[1])
            p_acc.append(r[2])
            t_acc.append(r[3])
    position_accuracies.append(p_acc)
    time_accuracies.append(t_acc)
    fed_weights = fed_avg(weights, num_samps)
    fed_model.model.set_weights(fed_weights)
    
    

  0%|          | 0/55 [00:00<?, ?it/s]

DAY0	current Time = 16:47:03


2023-02-03 16:47:12.787126: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


DAY1	current Time = 16:52:32
DAY2	current Time = 16:57:56
DAY3	current Time = 17:03:56
DAY4	current Time = 17:10:05
DAY5	current Time = 17:16:20
DAY6	current Time = 17:22:37
DAY7	current Time = 17:28:38
DAY8	current Time = 17:34:43
DAY9	current Time = 17:40:51
DAY10	current Time = 17:47:39
DAY11	current Time = 17:53:37
DAY12	current Time = 17:59:58
DAY13	current Time = 18:06:29
DAY14	current Time = 18:12:41
DAY15	current Time = 18:19:26
DAY16	current Time = 18:25:53
DAY17	current Time = 18:32:45
DAY18	current Time = 18:39:04
DAY19	current Time = 18:46:31
DAY20	current Time = 18:52:42
DAY21	current Time = 18:59:17
DAY22	current Time = 19:06:21
DAY23	current Time = 19:14:09
DAY24	current Time = 19:21:19
DAY25	current Time = 19:27:57
DAY26	current Time = 19:35:01
DAY27	current Time = 19:41:04
DAY28	current Time = 19:47:58
DAY29	current Time = 19:54:51
DAY30	current Time = 20:01:34
DAY31	current Time = 20:09:05
DAY32	current Time = 20:16:13
DAY33	current Time = 20:22:59
DAY34	current Time 

In [14]:
save = {
    "position_accuracies": position_accuracies,
    "time_accuracies": time_accuracies
}

In [15]:
with open("../../02_data/fed_accuracies2.json", "w") as f:
    json.dump(save, f)

In [16]:
fed_model.model.save("../../04_nn_models/fed_noshare2")

INFO:tensorflow:Assets written to: ../../04_nn_models/fed_noshare2/assets
